In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## 1 žingsnis: Apibrėžkite Pydantic modelius struktūrizuotiems rezultatams

Šie modeliai apibrėžia **schemą**, kurią agentai grąžins. Naudojant `response_format` su Pydantic užtikrinama:
- ✅ Tipų saugi duomenų ištrauka
- ✅ Automatinė validacija
- ✅ Jokios klaidos, susijusios su laisvo teksto atsakymais
- ✅ Lengvas sąlyginis maršrutizavimas pagal laukus


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## 2 žingsnis: Sukurkite viešbučių rezervavimo įrankį

Šis įrankis yra tas, kurį **availability_agent** naudos patikrinti, ar kambariai yra laisvi. Naudojame `@ai_function` dekoratorių, kad:
- Paverstume Python funkciją į AI naudojamą įrankį
- Automatiškai sugeneruotume JSON schemą LLM
- Atliktume parametrų validaciją
- Įgalintume automatinį agentų iškvietimą

Šiame demonstraciniame pavyzdyje:
- **Stokholmas, Sietlas, Tokijas, Londonas, Amsterdamas** → Yra laisvų kambarių ✅
- **Visi kiti miestai** → Nėra laisvų kambarių ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 3 žingsnis: Apibrėžkite sąlygų funkcijas maršrutizavimui

Šios funkcijos analizuoja agento atsakymą ir nustato, kuriuo keliu tęsti darbo eigą.

**Pagrindinis modelis:**
1. Patikrinkite, ar pranešimas yra `AgentExecutorResponse`
2. Išanalizuokite struktūrizuotą išvestį (Pydantic modelis)
3. Grąžinkite `True` arba `False`, kad valdytumėte maršrutizavimą

Darbo eiga įvertins šias sąlygas **kraštinėse**, kad nuspręstų, kurį vykdytoją iškviesti toliau.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## 4 žingsnis: Sukurkite pasirinktą rodymo vykdytoją

Vykdytojai yra darbo eigos komponentai, atliekantys transformacijas arba šalutinius poveikius. Naudojame dekoratorių `@executor`, kad sukurtume pasirinktą vykdytoją, kuris rodo galutinį rezultatą.

**Pagrindinės sąvokos:**
- `@executor(id="...")` - Registruoja funkciją kaip darbo eigos vykdytoją
- `WorkflowContext[Never, str]` - Tipų užuominos įvesties/išvesties duomenims
- `ctx.yield_output(...)` - Pateikia galutinį darbo eigos rezultatą


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## 5 žingsnis: Įkelkite aplinkos kintamuosius

Konfigūruokite LLM klientą. Šis pavyzdys veikia su:
- **GitHub modeliais** (nemokamas planas su GitHub tokenu)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## 6 žingsnis: Sukurkite AI agentus su struktūrizuotais rezultatais

Sukuriame **tris specializuotus agentus**, kiekvieną apgaubtą `AgentExecutor`:

1. **availability_agent** - Tikrina viešbučių užimtumą naudodamas įrankį
2. **alternative_agent** - Siūlo alternatyvius miestus (kai nėra laisvų kambarių)
3. **booking_agent** - Skatina užsakymą (kai yra laisvų kambarių)

**Pagrindinės savybės:**
- `tools=[hotel_booking]` - Suteikia agentui įrankį
- `response_format=PydanticModel` - Užtikrina struktūrizuotą JSON išvestį
- `AgentExecutor(..., id="...")` - Apgaubia agentą, kad būtų galima naudoti darbo eigoje


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## 7 žingsnis: Sukurkite darbo eigą su sąlyginėmis jungtimis

Dabar naudojame `WorkflowBuilder`, kad sukurtume grafiką su sąlyginiu maršrutavimu:

**Darbo eigos struktūra:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Pagrindiniai metodai:**
- `.set_start_executor(...)` - Nustato pradinį tašką
- `.add_edge(from, to, condition=...)` - Prideda sąlyginę jungtį
- `.build()` - Užbaigia darbo eigą


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## 8 žingsnis: Paleiskite testavimo atvejį 1 - Miestas BE prieinamumo (Paryžius)

Išbandykime **be prieinamumo** scenarijų, prašydami viešbučių Paryžiuje (kur mūsų simuliacijoje nėra laisvų kambarių).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## 9 žingsnis: Vykdykite 2 testavimo atvejį - Miestas SU prieinamumu (Stokholmas)

Dabar išbandykime **prieinamumo** kelią, prašydami viešbučių Stokholme (kur mūsų simuliacijoje yra laisvų kambarių).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Pagrindinės mintys ir tolimesni žingsniai

### ✅ Ką išmokote:

1. **WorkflowBuilder šablonas**
   - Naudokite `.set_start_executor()` norėdami nustatyti pradinį tašką
   - Naudokite `.add_edge(from, to, condition=...)` sąlyginiam maršrutizavimui
   - Iškvieskite `.build()` norėdami užbaigti darbo eigą

2. **Sąlyginis maršrutizavimas**
   - Sąlygų funkcijos tikrina `AgentExecutorResponse`
   - Analizuokite struktūrizuotus rezultatus, kad priimtumėte maršrutizavimo sprendimus
   - Grąžinkite `True`, kad aktyvuotumėte ryšį, `False`, kad jį praleistumėte

3. **Įrankių integracija**
   - Naudokite `@ai_function`, kad Python funkcijas paverstumėte AI įrankiais
   - Agentai automatiškai kviečia įrankius, kai jų reikia
   - Įrankiai grąžina JSON, kurį agentai gali analizuoti

4. **Struktūrizuoti rezultatai**
   - Naudokite Pydantic modelius, kad saugiai išgautumėte duomenis pagal tipus
   - Nustatykite `response_format=MyModel` kurdami agentus
   - Analizuokite atsakymus naudodami `Model.model_validate_json()`

5. **Individualūs vykdytojai**
   - Naudokite `@executor(id="...")`, kad sukurtumėte darbo eigos komponentus
   - Vykdytojai gali transformuoti duomenis arba atlikti šalutinius veiksmus
   - Naudokite `ctx.yield_output()`, kad pateiktumėte darbo eigos rezultatus

### 🚀 Praktinės taikymo sritys:

- **Kelionių rezervacija**: Tikrinkite prieinamumą, siūlykite alternatyvas, lyginkite pasirinkimus
- **Klientų aptarnavimas**: Maršrutizuokite pagal problemos tipą, nuotaiką, prioritetą
- **Elektroninė prekyba**: Tikrinkite inventorių, siūlykite alternatyvas, apdorokite užsakymus
- **Turinio moderavimas**: Maršrutizuokite pagal toksiškumo balus, vartotojų žymes
- **Patvirtinimo procesai**: Maršrutizuokite pagal sumą, vartotojo rolę, rizikos lygį
- **Daugiapakopis apdorojimas**: Maršrutizuokite pagal duomenų kokybę, pilnumą

### 📚 Tolimesni žingsniai:

- Pridėkite sudėtingesnes sąlygas (kelis kriterijus)
- Įgyvendinkite ciklus su darbo eigos būsenos valdymu
- Pridėkite sub-darbo eigas, kad sukurtumėte pakartotinai naudojamus komponentus
- Integruokite su realiais API (viešbučių rezervacija, inventoriaus sistemos)
- Pridėkite klaidų tvarkymą ir atsarginius kelius
- Vizualizuokite darbo eigas naudodami integruotus vizualizavimo įrankius



---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar neteisingus aiškinimus, kylančius dėl šio vertimo naudojimo.
